# PHYS3030 Electromagnétisme [J. Martin & J. Denis]
## Répétition Calculs

On compile tout d'abord les packages qui vont nous être utiles par la suite.

**ForwardDiff** : Permet de calculer le gradient d'une fonction à l'aide de la [dérivation automatique](https://fr.wikipedia.org/wiki/D%C3%A9rivation_automatique).

**LinearAlgebra** : Permet d'utiliser des fonctions classiques d'algèbre linéaire.

**LaTeXStrings** : Permet d'utiliser LaTeX dans les figures.

**GLMakie** : Package complet de génération de figures interactives en 2D ou 3D. C'est un sous-package de [Makie](https://makie.juliaplots.org/stable/). Un site reprenant de nombreux exemples de figures réalisées avec Makie peut être trouvé [ici](https://lazarusa.github.io/BeautifulMakie/).

In [ ]:
# Installation des packages
import Pkg; 
Pkg.add("ForwardDiff");
Pkg.add("LinearAlgebra");
Pkg.add("LaTeXStrings");
Pkg.add("GLMakie");

In [2]:
# Lancement des packages
using ForwardDiff
using LinearAlgebra
using LaTeXStrings
using GLMakie

En Julia, le point-virgule à la fin d'une ligne de code est optionnel. Il permet simplement d'indiquer si on veut afficher ou non le résultat de la ligne.

## Exercice C7

**Enoncé** : Calculez la dérivée du champ scalaire $V(\mathbf{r})=x \, e^{-(y^2+x^2)}$ dans les directions $\mathbf{n}=\frac{1}{3}(1,1,1)$ et $\mathbf{n'}=\cos\varphi\, \mathbf{e}_x + \sin\varphi \, \mathbf{e}_y$.

**Code** : Définissons d'abord le champ scalaire et les vecteurs $\mathbf{n}$ et $\mathbf{n'}$ et calculons le gradient du champ à l'aide de ForwardDiff. En Julia, on peut donner plusieurs définitions à une même fonction. On voit ainsi que l'on peut définir le champ $V$ soit avec les deux variables cartésiennes $x$ et $y$ soit avec le vecteur position $\mathbf{r}$. A la compilation, le programme détecte automatiquement quelle définition appeler sur base des arguments que l'on donne à la fonction.

In [3]:
# Définitions
V(x,y) = x*exp(-(x^2+y^2))
V(r) = V(r[1],r[2])
n1(x,y) = 1/√3*[1.,1.]
n2(x,y) = [-sin(atan(y,x)),cos(atan(y,x))]

# Calcul du gradient
∇V(r) = ForwardDiff.gradient(V,r)

# Test
println(V(2,1))
println(V([2,1]))
println(∇V([2,1]))

0.013475893998170934
0.013475893998170934
[-0.04716562899359827, -0.026951787996341868]


La dérivée d'un champ scalaire dans une certaine direction est donnée par le produit scalaire du gradient de la fonction avec le vecteur direction. Pour faire le produit scalaire, on utilise la fonction dot du package LinearAlgebra.

In [4]:
# Dérivées de V dans les direction n et n'
n1_∇V(x,y) = dot(∇V([x,y]),n1(x,y))
n2_∇V(x,y) = dot(∇V([x,y]),n2(x,y))

# Test
println(n1_∇V(2,1))
println(n2_∇V(2,1))

-0.04279171065078169
-0.003013301503749164


Visualisons à présent le résultat avec GLMakie. Pour cela, il faut d'abord initialiser une figure dont on peut préciser la résolution. Au passage, on passe le thème de Makie en noir parce que c'est plus joli. On définit aussi les couleurs que l'on va utiliser avec entre autre la fonction cgrad qui permet de manipuler les [gradients de couleurs de Makie](https://makie.juliaplots.org/stable/documentation/colors/index.html).

In [5]:
set_theme!(theme_black())
set_window_config!(vsync = false)
fig = Figure(resolution=(1200,700));
cmap = :haline
cmap2 = cgrad(:ice, 1.0:-0.01:0.0, categorical = true)[50:80];

Makie est très efficace pour afficher plusieurs graphes sur une même figure. Pour cela, il suffit de définir plusieurs axes et de préciser la position de chaque axe sur la figure. On peut en même temps donner un titre à chaque graphe (en LaTeX avec le L devant le titre) et nommer chacun des axes. Pour une figure en 3D, il faut utiliser Axis3 pour avoir un système d'axes à trois dimensions.

In [6]:
ax11 = Axis(fig[1,1],xlabel="x",ylabel="y",title=L"Gradient de $V$")
ax21 = Axis3(fig[2,1],title=L"$V$")
ax12 = Axis(fig[1,2],xlabel="x",ylabel="y",title=L"Champ de vecteur $\mathbf{n}$")
ax22 = Axis3(fig[2,2],title=L"Dérivée de $V$ dans la direction $\mathbf{n}$")
ax13 = Axis(fig[1,3],xlabel="x", ylabel="y",title=L"Champ de vecteur $\mathbf{n'}$")
ax23 = Axis3(fig[2,3],title=L"Dérivée de $V$ dans la direction $\mathbf{n'}$");

Il ne reste plus qu'à préciser quels graphes nous voulons sur quels axes. Nous pouvons afficher la surface d'un champ scalaire à l'aide de la fonction [surface!](https://makie.juliaplots.org/stable/examples/plotting_functions/surface/index.html) et un champ de vecteur à l'aide de [streamplot!](https://makie.juliaplots.org/stable/examples/plotting_functions/streamplot/index.html). Attention que streamplot demande que la fonction en entrée renvoie un Point. Le point d'exclamation ! à la fin du nom de la fonction permet de dire à Makie qu'il faut ajouter ce graphe à la figure et non créer une nouvelle figure.

In [7]:
# On définit les limites des graphes
xs = LinRange(-2, 2, 1000)
ys = LinRange(-2, 2, 1000)

# Axe en haut à gauche, gradient de V
stream1(x,y) = Point(∇V([x,y])...)
streamplot!(ax11,stream1,-2..2,-2..2,colormap=cmap)

# Axe en bas à gauche, surface de V ainsi que son gradient en dessous
surface!(ax21,xs, ys, V, colormap=cmap);
minField = minimum(V(x,y) for x in xs for y in ys)
streamplot!(ax21,stream1,-2..2,-2..2,transformation = (:xy, minField),colormap=cmap)

# Axe en haut au milieu, champ de vecteur n
stream2(x,y) = Point(n1(x,y)...)
streamplot!(ax12,stream2,-1..1,-1..1,arrow_size=15,density=0.7,colormap=cmap2)

# Axe en bas au milieu, dérivée de V dans la direction n
surface!(ax22,xs,ys,n1_∇V,colormap=cmap)

# Axe en haut à droite, champ de vecteur n'
stream3(x,y) = Point(n2(x,y)...)
streamplot!(ax13,stream3,-1..1,-1..1,arrow_size=15,density=0.7,colormap=cmap2)

# Axe en bas à droite, dérivée de V dans la direction n'
surface!(ax23,xs,ys,n2_∇V,colormap=cmap);

Les trois points ... après un vecteur est utilisé lors de l'appel à une fonction pour développer le vecteur de tel sorte que chacun de ses éléments est considéré comme un argument pour la fonction. Ainsi Point(n1(x,y)...) est similaire à Point(n1(x,y)[1],n1(x,y)[2]).

Il ne reste plus à présent qu'à afficher la figure !

In [8]:
display(fig)

GLMakie.Screen(...)

## Exercice C8

**Enoncé** : Dans une région de l'espace règne le potentiel électrostatique
\begin{equation*}
    V(x,y) = 10(2xy - 3x^2 - 4y^2 - 18x + 28y + 12)
\end{equation*}
Déterminez la position et la hauteur du maximum de potentiel. Déterminez ensuite la direction et le module de la plus grande pente au point $(1,1)$.

**Code** : La position du maximum d'un champ scalaire peut être déterminé en trouvant les points d'annulation du gradient de ce champ. De même, la plus grande pente en un point d'un champ scalaire est donné par le gradient en ce point. Définissons donc $V$ et son gradient.

In [9]:
V(x,y) = 10(2x*y - 3x^2 - 4y^2 - 18x + 28y + 12)
V(r) = V(r[1],r[2])
∇V(r) = ForwardDiff.gradient(V,r);

Affichons la surface du champ scalaire et son gradient sur deux panneaux.

In [10]:
fig = Figure(resolution=(1200,700));
ax1 = Axis3(fig[1, 1], title=L"V")
xs = LinRange(-20,20, 1000)
ys = LinRange(-20,20, 1000)
ax2 = Axis(fig[1,2],xlabel="x",ylabel="y",title=L"Gradient de $V$")

minField = minimum(V(x,y) for x in xs for y in ys)
stream(x,y) = Point(∇V([x,y])...)
streamplot!(ax1,stream,-20..20,-20..20,transformation = (:xy, minField))
streamplot!(ax2,stream,-20..20,-20..20)
surfacePoints = [V(x,y) for x in xs, y in ys]
surface!(ax1,xs, ys, surfacePoints, colormap=:viridis)
display(fig)

GLMakie.Screen(...)

En utilisant certaines fonctions avancées de GLMakie comme les sliders, on peut aussi ajouter des paramètres au champ que l'on peut ensuite manipuler en temps réel. Tout ceci est défini dans la fonction ScalarField qui fonctionne pour n'importe quel champ scalaire (qui se comporte bien).

In [11]:
function ScalarField(V,xlims=[-3,3]::AbstractArray, ylims=[-3,3]::AbstractArray, steps=100::Int;sliderLimits=[]::Vector)
    set_theme!(theme_black())
    nSliders = length(sliderLimits)
    local sliders,sliderobservables,surfacePoints

    fig = Figure();
    ax1 = Axis3(fig[1, 1])
    xs = LinRange(xlims..., steps)
    ys = LinRange(ylims..., steps)

    if nSliders > 0
        sliders = [Slider(fig[1,i+1], range=sliderLimits[i], horizontal=false, height=350, label=string(i)) for i in 1:nSliders]
        [Label(fig[1,i+1],string(i),color=:black) for i in 1:nSliders]
        sliderobservables = [s.value for s in sliders]
        for i in 1:nSliders
            set_close_to!(sliders[i],(sliderLimits[i][end]-sliderLimits[i][1])/2)
        end
        surfacePoints = lift(sliderobservables...) do a...
            [V(x,y,a...) for x in xs, y in ys]
        end
    else
        ∇V(r) = ForwardDiff.gradient(V,r)
        ax2 = Axis(fig[1,2],xlabel="x",ylabel="y")
        minField = minimum(V(x,y) for x in xs for y in ys)
        surfacePoints = [V(x,y) for x in xs, y in ys]
        stream(x,y) = Point(∇V([x,y])...)
        streamplot!(ax1,stream,xlims[1]..xlims[2],ylims[1]..ylims[2],transformation = (:xy, minField))
        streamplot!(ax2,stream,xlims[1]..xlims[2],ylims[1]..ylims[2])
    end

    surface!(ax1,xs, ys, surfacePoints, colormap=:viridis)
    display(fig)
end;

Comme exemple, on ajoute ici des paramètres $a$, $b$ et $c$ comme facteurs sur les termes intéressants du champ scalaire. On peut ainsi obtenir un point de selle où le gradient s'annule mais ce point est à la fois minimal et maximal selon la direction.

In [12]:
V(x,y,a,b,c) = 10(c*x*y - a*x^2 - b*y^2 - 18x + 28y + 12)
ScalarField(V,[-20,20],[-20,20],sliderLimits=[-6:0.01:6,-8:0.01:8,-4:0.01:4])

GLMakie.Screen(...)

## Exercice C12

**Enoncé** : Montrez que le champ
\begin{equation*}
    \mathbf{A} = \frac{\mathbf{e}_\varphi}{\rho} = \left( \frac{-y}{x^2+y^2}, \frac{x}{x^2+y^2},0 \right)
\end{equation*}
est simultanément indivergentiel et irrotationnel $\forall \rho \neq 0$, mais que la circulation le long d'une boucle entourant l'axe $\mathbf{e}_z$ est non nulle. Calculez celle-ci pour un tour complet dans le sens trigonométrique autour de $\mathbf{e}_z$.

**Code** : La paramétrisation d'une boucle entourant l'axe $\mathbf{e}_z$ de rayon $R$ se situant à une hauteur $z$ est $\mathbf{r}(\varphi)=R \, \mathbf{e}_\rho + z \mathbf{e}_z$. La circulation de $\mathbf{A}$ le long du chemin $\mathcal{C}$ du cercle est
\begin{equation*}
    \int_\mathcal{C} \mathbf{A} \cdot d\mathbf{r} = \int_{\varphi_i}^{\varphi_f} \mathbf{A} \cdot \frac{d\mathbf{r}}{d\varphi} d\varphi
\end{equation*}
Définissons donc le champ vectoriel $\mathbf{A}$, la paramétrisation du cercle $\mathbf{r} = R(\cos \varphi \, \mathbf{e}_x + \sin \varphi \, \mathbf{e}_y) + z\, \mathbf{e}_z$ et $d\mathbf{r}/d\varphi = R(-\sin \varphi \, \mathbf{e}_x + \cos \varphi \, \mathbf{e}_y)$.

In [13]:
R = 0.3
z = 3

A(x,y,z) = [-y/(x^2+y^2),x/(x^2+y^2),0]
r(φ) = [R*cos(φ),R*sin(φ),z]
dr(φ) = [-R*sin(φ),R*cos(φ),0];

On initialise la figure ainsi que les axes.

In [14]:
cmap = :viridis
fig = Figure(resolution=(1400,700), fontsize=20);
ax1 = Axis3(fig[1,1],xlabel="x",ylabel="y",title=LaTeXString("Contour du cercle et sa paramétrisation"))
ax2 = Axis(fig[1,2],xlabel="x",ylabel="y",title=L"Champs vectoriels $\mathbf{A}$ et $\frac{d\mathbf{r}}{dφ}$");

Sur le panneau de gauche, on représente le cercle ainsi que sa paramétrisation. Pour cela, on utilise la fonction [lines!](https://makie.juliaplots.org/stable/examples/plotting_functions/lines/index.html) qui relie les points qu'on lui passe en argument et la fonction [arrows!](https://makie.juliaplots.org/stable/examples/plotting_functions/arrows/index.html) qui dessine des vecteurs dont on doit spécifier l'origine et la direction. Sur le panneau de droite, on représente le champ de vecteur $\mathbf{A}$, le cercle ainsi que $d\mathbf{r}/d\varphi$. On ajoute aussi tout à droite une barre de couleur à l'aide de [Colorbar](https://makie.juliaplots.org/stable/examples/layoutables/colorbar/index.html) qui permet de quantifier la norme du champ $\mathbf{A}$ à l'aide de ses couleurs.

In [15]:
# Champ de vecteur F
stream(x,y) = Point2(A(x,y,0)[1:2])
streamplot!(ax2,stream,-0.5..0.5,-0.5..0.5,colormap=cmap,arrow_size=20,linewidth=3)

# Contour du cercle
rs = [r(t) for t in 0:pi/50:(2+1/50)*pi]
rs_x = [rs[i][1] for i in 1:length(rs)]
rs_y = [rs[i][2] for i in 1:length(rs)]
rs_z = [rs[i][3] for i in 1:length(rs)]
lines!(ax1,rs_x,rs_y, rs_z, linewidth=4)
lines!(ax2,rs_x,rs_y, linewidth=3)

# Paramétrisation du cercle
rs = [r(t) for t in 0:pi/10:2*pi]
norms = norm.(rs)
rs_x = [rs[i][1] for i in 1:length(rs)]
rs_y = [rs[i][2] for i in 1:length(rs)]
rs_z = [rs[i][3] for i in 1:length(rs)]
arrows!(ax1, zeros(length(rs_x)), zeros(length(rs_y)), zeros(length(rs_z)), rs_x, rs_y, rs_z, arrowsize = Vec3f(0.03, 0.03, 0.25), lengthscale = 1.0, linewidth=0.004, arrowcolor = :lightblue, linecolor = :lightblue)

# Champ de vecteur dr/dt
positions_x = [r(t)[1] for t in 0:pi/8:2*pi]
positions_y = [r(t)[2] for t in 0:pi/8:2*pi]
drs = [dr(t) for t in 0:pi/8:2*pi]
norms = norm.(drs)
drs_x = [drs[i][1] for i in 1:length(drs)]
drs_y = [drs[i][2] for i in 1:length(drs)]
arrows!(ax2,positions_x, positions_y, drs_x, drs_y, arrowsize = 20, lengthscale = 0.3, arrowcolor = :lightblue, linecolor = :lightblue)

# Barre de couleur
norms = [norm(A(x,y,0)) for x in 0.001:0.001:0.5 for y in 0.001:0.001:0.5]
Colorbar(fig[1,3], limits =(minimum(norms), maximum(norms)), nsteps =100, colormap = cmap, ticksize=15, width = 15, tickalign=1)

display(fig)

GLMakie.Screen(...)

# Exercice C19

**Enoncé** : Montrez que l'ellipse d'équation $x^2/a^2+y^2/b^2=1$ peut être paramétrée par le chemin
\begin{equation*}
    \mathbf{r}(t) = (a \sin t,b \cos t), \qquad t \in [0,2\pi]
\end{equation*}
Soit le champ vectoriel $\mathbf{F}=y(4x^2+y^2) \, \mathbf{e}_x + x(2x^2+3y^2) \, \mathbf{e}_y$. Calculez sa circulation le long de l'ellipse parcourue dans le sens trigonométrique. Si l'ellipse est parcourue dans le sens horloger, que vaut alors la circulation ?

**Code** : La circulation de $\mathbf{F}$ le long du chemin $\mathcal{C}$ de l'ellipse est
\begin{equation*}
    \int_\mathcal{C} \mathbf{F} \cdot d\mathbf{r} = \int_{t_i}^{t_f} \mathbf{F} \cdot \frac{d\mathbf{r}}{dt} dt
\end{equation*}
Définissons donc le champ vectoriel $\mathbf{F}$, la paramétrisation $\mathbf{r}$ de l'ellipse et $d\mathbf{r}/dt = (a \cos t, -b \sin t)$. 

In [16]:
a = 0.3;
b = 0.15;

F(x,y) = [y*(4x^2+y^2),x*(2x^2+3y^2)]
r(t) = [a*sin(t),b*cos(t)]
dr(t) = (a*cos(t),-b*sin(t));

On initialise la figure et les axes en précisant leurs limites.

In [17]:
cmap = :viridis
fig = Figure(resolution=(1400,700), fontsize=20);
ax1 = Axis(fig[1,1],xlabel="x",ylabel="y",title=LaTeXString("Contour de l'ellipse et sa paramétrisation"))
ax2 = Axis(fig[1,2],xlabel="x",ylabel="y",title=L"Champs vectoriels $\mathbf{F}$ et $\frac{d\mathbf{r}}{dt}$")
xlims!(ax1,-0.4,0.4)
ylims!(ax1,-0.4,0.4);

Dans le panneau de gauche, on représente l'ellipse ainsi que sa paramétrisation $\mathrm{r}$. Dans le panneau de droite, on dessine le champ de vecteur $\mathbf{F}$ ainsi que l'ellipse et $d\mathbf{r}/dt$. On ajoute tout à droite une barre de couleur pour quantifier la norme de $\mathbf{F}$ à partir de ses couleurs.

In [19]:
# Champ de vecteur F
stream(x,y) = Point2(F(x,y))
streamplot!(ax2,stream,-0.5..0.5,-0.5..0.5,colormap=cmap,arrow_size=20,linewidth=3)

# Contour de l'ellipse
rs = [r(t) for t in 0:pi/50:(2+1/50)*pi]
rs_x = [rs[i][1] for i in 1:length(rs)]
rs_y = [rs[i][2] for i in 1:length(rs)]
lines!(ax1,rs_x,rs_y, linewidth=3)
lines!(ax2,rs_x,rs_y, linewidth=3)

# Paramétrisation de l'ellipse
rs = [r(t) for t in 0:pi/10:2*pi]
norms = norm.(rs)
rs_x = [rs[i][1] for i in 1:length(rs)]
rs_y = [rs[i][2] for i in 1:length(rs)]
arrows!(ax1,zeros(length(rs_x)), zeros(length(rs_y)), rs_x, rs_y, arrowsize = 20, lengthscale = 1.0, arrowcolor = :lightblue, linecolor = :lightblue)

# Champ de vecteur dr/dt
positions_x = [r(t)[1] for t in 0:pi/8:2*pi]
positions_y = [r(t)[2] for t in 0:pi/8:2*pi]
drs = [dr(t) for t in 0:pi/8:2*pi]
norms = norm.(drs)
drs_x = [drs[i][1] for i in 1:length(drs)]
drs_y = [drs[i][2] for i in 1:length(drs)]
arrows!(ax2,positions_x, positions_y, drs_x, drs_y, arrowsize = 20, lengthscale = 0.3, arrowcolor = :lightblue, linecolor = :lightblue)

# Barre de couleur
norms = [norm(F(x,y)) for x in -0.5:0.001:0.5 for y in -0.5:0.001:0.5]
Colorbar(fig[1,3], limits =(minimum(norms), maximum(norms)), nsteps =100, colormap = cmap, ticksize=15, width = 15, tickalign=1)

display(fig)

GLMakie.Screen(...)